In [1]:
%matplotlib widget

In [2]:
import functools
import numpy as np
from beam.system import System
from beam import mesh
from beam import postprocessing as postproc
import matplotlib.pyplot as plt

In [3]:
def case(nel):
    """
    In this example, a ring is twisted. A ring is clamped at some point
    and a twisting moment is applied at the opposite point.
    All elements are non-mortar.
    Contact, static analysis.
    """
    
    mat = {
        'EA':2.76461e3,
        'GA1':1.03924e3,
        'GA2':1.03924e3,
        'GIt':2078.5,
        'EI1':2764.52,
        'EI2':2764.52,
        'Arho':1.0,
        'I12rho':1.0,
        'I1rho':1.0,
        'I2rho':1.0,
        'Contact radius':0.02 * 2 * np.pi
    }

    (coordinates, elements) = mesh.circle_mesh(
        R=1, n_elements=nel, order=2, material=mat,
        reference_vector=np.array([0,0,1]), plane=(0,1),
        starting_node_index=0
    )

    exclude_range = 1

    for i in range(len(elements)):
        mortar_elements = elements.copy()
        exclude_indices = []
        for j in range(-exclude_range, exclude_range+1):
            exclude_indices.append((i+j)%len(elements))
        exclude_indices.sort(reverse=True)
        for j in exclude_indices:
            mortar_elements.pop(j)

        mesh.add_mortar_element(
            [elements[i]],
            possible_contact_partners=mortar_elements.copy(),
            n_contact_integration_points=10
        )

    system = System(coordinates, elements)
    system.time_step = 1.0
    system.max_number_of_time_steps = 1000
    system.max_number_of_contact_iterations = 10
    system.final_time = 14.0
    system.solver_type = 'static'
    system.convergence_test_type = 'RES'
    system.contact_detection = True
    system.print_residual = True
    system.tolerance = 1e-6
    system.printing = False
    
    def user_force_load(self):
        n_nodes = self.get_number_of_nodes()
        Q = np.zeros((6, n_nodes))
        if self.current_time < 9:
            Q[3,n_nodes//2] = 700*self.current_time
        else:
            Q[3,n_nodes//2] = 700*9 + 70*(self.current_time - 9)
        return Q
    
    system.degrees_of_freedom[-1][:6,0] = False  # [current time, dof 0 through 5, first node of the first beam]
    system.degrees_of_freedom[-1][6,:] = False
    system.force_load = functools.partial(user_force_load, system)
    
    return system

In [4]:
system1 = case(8)
system1.solve()

In [5]:
system2 = case(32)
system2.solve()

In [6]:
postproc.line_plot(system1, (-1,1), (-1,1), (-1,1), 14)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [30]:
plt.rcParams.update({
    "text.usetex": True,
    "font.family": "sans-serif",
    "font.sans-serif": ["Helvetica"],
    "font.size": 20
})
fig = plt.figure(figsize=(6,5), tight_layout=True)
ax = plt.axes()
ax.plot(*system1.contact_force_function(-1).T, label='8 elements', linewidth=3.0)
ax.plot(*system2.contact_force_function(-1).T, '--', label='32 elements', linewidth=3.0)
ax.set_xlim((0,2*np.pi))
ax.set_xlabel('$s$')
ax.set_ylabel('$\lambda$')
plt.legend()
#plt.savefig("force.pdf", bbox_inches='tight')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [9]:
plt.rcParams.update({
    "text.usetex": True,
    "font.family": "sans-serif",
    "font.sans-serif": ["Helvetica"],
    "font.size": 20
})
fig = plt.figure(figsize=(6,5), tight_layout=True)
ax = plt.axes()
ax.scatter(*system2.gap_function[-1].T, marker='.', s=100, label='32 elements')
ax.scatter(*system1.gap_function[-1].T, marker='.', s=100, label='8 elements')
ax.set_xlim((0,2*np.pi))
ax.set_xlabel('$s$')
ax.set_ylabel('$g$')
plt.legend()
#plt.savefig("gap.pdf", bbox_inches='tight')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [10]:
def case2(nel):
    """
    In this example, a ring is twisted. A ring is clamped at some point
    and a twisting moment is applied at the opposite point.
    Only first half of elements are non-mortar.
    Contact, static analysis.
    """
    
    mat = {
        'EA':2.76461e3,
        'GA1':1.03924e3,
        'GA2':1.03924e3,
        'GIt':2078.5,
        'EI1':2764.52,
        'EI2':2764.52,
        'Arho':1.0,
        'I12rho':1.0,
        'I1rho':1.0,
        'I2rho':1.0,
        'Contact radius':0.02 * 2 * np.pi
    }

    (coordinates, elements) = mesh.circle_mesh(
        R=1, n_elements=nel, order=2, material=mat,
        reference_vector=np.array([0,0,1]), plane=(0,1),
        starting_node_index=0
    )

    exclude_range = 1

    for i in range(len(elements)//2):
        mortar_elements = elements.copy()
        exclude_indices = []
        for j in range(-exclude_range, exclude_range+1):
            exclude_indices.append((i+j)%len(elements))
        exclude_indices.sort(reverse=True)
        for j in exclude_indices:
            mortar_elements.pop(j)

        mesh.add_mortar_element(
            [elements[i]],
            possible_contact_partners=mortar_elements.copy(),
            n_contact_integration_points=10
        )

    system = System(coordinates, elements)
    system.time_step = 1.0
    system.max_number_of_time_steps = 1000
    system.max_number_of_contact_iterations = 10
    system.final_time = 14.0
    system.solver_type = 'static'
    system.convergence_test_type = 'RES'
    system.contact_detection = True
    system.print_residual = True
    system.tolerance = 1e-6
    system.printing = False

    def user_force_load(self):
        n_nodes = self.get_number_of_nodes()
        Q = np.zeros((6, n_nodes))
        if self.current_time < 9:
            Q[3,n_nodes//2] = 700*self.current_time
        else:
            Q[3,n_nodes//2] = 700*9 + 70*(self.current_time - 9)
        return Q
    
    system.degrees_of_freedom[-1][:6,0] = False  # [current time, dof 0 through 5, first node of the first beam]
    system.degrees_of_freedom[-1][6,:] = False
    system.force_load = functools.partial(user_force_load, system)
    
    return system

In [11]:
system12 = case2(8)
system12.solve()

In [25]:
plt.rcParams.update({
    "text.usetex": True,
    "font.family": "sans-serif",
    "font.sans-serif": ["Helvetica"],
    "font.size": 20
})
fig = plt.figure(figsize=(6,5), tight_layout=True)
ax = plt.axes()
ax.plot(*system1.contact_force_function(-1).T, label='8 non-mortar', linewidth=3.0)
ax.plot(*system12.contact_force_function(-1).T, '--', label='4 non-mortar', linewidth=3.0)
ax.set_xlim((0,2*np.pi))
ax.set_xlabel('$s$')
ax.set_ylabel('$\lambda$')
plt.legend()
#plt.savefig("force.pdf", bbox_inches='tight')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [21]:
plt.rcParams.update({
    "text.usetex": True,
    "font.family": "sans-serif",
    "font.sans-serif": ["Helvetica"],
    "font.size": 20
})
fig = plt.figure(figsize=(6,5), tight_layout=True)
ax = plt.axes()
ax.scatter(*system1.gap_function[-1].T, marker='.', s=100, label='8 non-mortar')
ax.scatter(*system12.gap_function[-1].T, marker='x', s=100, label='4 non-mortar')
ax.set_xlim((0,2*np.pi))
ax.set_xlabel('$s$')
ax.set_ylabel('$g$')
plt.legend()
#plt.savefig("gap.pdf", bbox_inches='tight')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …